# Estimador de razón con muestreo estratificado
    - Existen dos técnicas generales para estimar la razón (R), emplear la razón por separado para cada estrato y emplear la razón de manera combinada.
    
    - Sirve para reducir el intervalo de confianza de los estimadores y mejorar la precisión

### Conclusiones
    - Para hacer las estimaciones de razón en estratos, se supone un m.a.s. dentro de cada estrato y no un m.a.s.  generalizado para el cuál se quieran mejorar los estimadores con estratos y con razón simultáneamente.
    
    - Estimar la razón por separado con muestras pequeñas, implica tener intervalos de confianza muy amplios
    
    - El estimador separado es una suma de cocientes, mientras que el combinado es un cociente de sumas

### 1] Definir una población
    - Notación:
        L: Número de estratos
        N: Tamaño poblacional
        N_h: Unidades poblacionales dentro de cada estrato
        Y[h,i]: Observación en el estrato h de la unidad i
    
    - Los parámetros como la media, varianza y total se calculan por separado para cada estrato como si fueran una población por separado. Para los cálculos poblacionales, los elementos son considerados con el mismo peso de importancia.

In [6]:
import numpy as np
import pandas as pd
import seaborn as sns

# Define the strata
# ... 600 is the total population, while the other elements in the list
# ... represent the units within each strata
# ... sample size is initially defined only for m.a.s.
L = 3
N = [900, 600, 200, 100]
n = [50, 0, 0, 0]
R = [0, 6, 5, 4]
confidence = 0.95

# Create DF outlining the strata for each unit
strata = np.repeat([1,2,3], N[1:])
df_population = pd.DataFrame(strata, columns=["strata"])

# Define score variable
km = np.random.normal(loc=21 , scale=1.5, size=N[0])
km = np.where(km<1, 1, km)
df_population["km"] = km

# Define ratio - distance/time
R_dict = {i: R[i] for i in range(1,4)}

# Define money variable
df_population["min"] = (df_population["km"] * (df_population["strata"].map(R_dict) +
                        np.random.normal(loc=0, scale=1.0, size=N[0])))

# Population Ratio
R[0] = df_population["min"].mean() / df_population["km"].mean()

# Output
print("="*100)
print("PARÁMETROS POBLACIONALES\n")
print("La razón min/km es:", round(R[0],2))
print("La media de tiempo es: ", round(df_population["min"].mean(axis=0),2))
print("El total de minutos es: ", int(df_population["min"].sum(axis=0)))
print("="*100)

PARÁMETROS POBLACIONALES

La razón min/km es: 5.54
La media de tiempo es:  116.08
El total de minutos es:  104470


### 2] Estimaciones con m.a.s.
    - Se realiza para comparar la mejora en la estimación estratificada empleando razón

In [7]:
from scipy.stats import t

# Select a RSS
df_sample = df_population.sample(n=n[0], random_state=100)

# Compute estimation of ratio
r = df_sample["min"].sum() / df_sample["km"].sum()
r_var = (1-n[0]/N[0]) * 1/n[0] * 1/df_sample["km"].mean(0)**2 * 1/(n[0]-1) * sum(
           (df_sample["min"] - r*df_sample["km"])**2)
r_std = r_var**0.5
r_interval = r + r_std*t(df=n[0]-1).ppf((0.5-confidence/2, 0.5+confidence/2))

# Compute estimation of mean
mean = df_sample["min"].mean(0)
var = df_sample["min"].var(0, ddof=1)
mean_var = (1-n[0]/N[0]) * var/n[0]
mean_std = mean_var**0.5
mean_interval = mean + mean_std*t(df=n[0]-1).ppf((0.5-confidence/2, 0.5+confidence/2))

# Compute estimation of total
total = N[0] * df_sample["min"].mean()
var = df_sample["min"].var(ddof=1)
total_var = N[0]**2 * (1-n[0]/N[0]) * var/n[0]
total_std = total_var**0.5
total_interval = total + total_std*t(df=n[0]-1).ppf((0.5-confidence/2, 0.5+confidence/2))

# Show results
print("="*100)
print("ESTIMACIÓN CON M.A.S.\n")
print("La estimación de la razón de min/km es:",round(r,2))
print("Se estima entre:",np.round(r_interval,2))
print("\nLa estimación de la media de minutos es:",round(mean,2))
print("Se estima entre:",np.round(mean_interval,2))
print("\nLa estimación del total de minutos es:",round(total,2))
print("Se estima entre:",np.round(total_interval,2))
print("="*100)

ESTIMACIÓN CON M.A.S.

La estimación de la razón de min/km es: 5.73
Se estima entre: [5.35 6.1 ]

La estimación de la media de minutos es: 119.79
Se estima entre: [112.08 127.5 ]

La estimación del total de minutos es: 107809.8
Se estima entre: [100872.71 114746.88]


### 3] Estimador de razón por separado
    - Se utiliza cuando se tienen pocos estratos y/o los tamaños de muestra en cada estrato son grandes.
    
    - Supone que las razones en cada estrato no son similares
    
    - Los sesgos de los estimadores de razón en cada estrato se suman, por lo que las varianzas suelen ser muy grandes. Por eso se indican cuando n_h son grandes
    
    - La razón de la muestra es la suma ponderada de las razones en cada estrato
    
        r = sum[for h in range(L)](N_h/N * r_h)
        r = sum[h in L](N_h/N * y_mean/x_mean)
            donde:
            r: Razón de la muestra completa
            r_h: Razón del estrato
            y_mean: Media de la variable objetivo
            x_mean: Media de la variable auxiliar
            
    - La varianza del estimador de razón se calcula como:
        Var(r) = sum[h in L](N_h^2/N^2 * Var(r_h))
            donde:
            Var(r): Varianza del estimador de razón de la muestra completa
            Var(r_h): Varianza del estimador de razón dentro del estrato h
            
    - La media se estima con:
        mean_s = sum[for h in L](N_h/N_0 * r_h*X_mean_h)
            donde:
            mean_s: Estimación de la media poblacional
            r_h: Estimación de la razón para el estrato h
            X_mean_h: Media poblaciónal dentro del estrato de la variable auxiliar
            
    - Con varianza:
        Var(mean) = sum[for h in L]( (N_h/N_0)^2 * X_mean_h^2 * Var(r_h) )
            donde:
            Var(mean): Varianza de la estimación de la media poblacional
            Var(r_h): Varianza del estimador de la razón para el estrato h
            
    - El total se estima como:
        total_s = sum[for h in L](r_h * X_h)
            donde:
            total_s: Estimación del total poblacional
            r_h: Estimación de la razón en cada estrato
            X_h: Total poblacional dentro de cada estrato para la variavle auxiliar
            
    - Con varianza:
        Var(total) = sum[for h in L](X_h^2 * Var(r_h))
            donde:
            Var(total): Varianza del estimador del total
            X_h: Total población dentro del estrato para la variable auxiliar
            Var(r_h): Varianza del estimador de la razón para el estrato h

In [8]:
# Analysis done using "min" as objective variable and "km" as ausiliary variable
from scipy.stats import norm

# Select a Stratified Random Sample (SRS)
# ... which is the same as a Random Simple Sample (RSS) within each strata
df_sample = (df_population.groupby("strata", group_keys=False)
             .apply(lambda x: x.sample(frac=n[0]/N[0], random_state=100)))

# Series with strata in sample
strata = df_sample["strata"]

# Define sample sizes
n[1:4] = df_sample["strata"].value_counts()



# Estimate Ratio for each strata
r = [0, 0, 0, 0]
for h in range(1,L+1):
    r[h] = df_sample["min"][strata==h].mean(axis=0) / df_sample["km"][strata==h].mean(axis=0)
    
# Estimate Ratio for the population
r[0] = sum(n[h]/n[0] * r[h] for h in range(1,L+1))

# Estimate variance of ratio estimator
x_mean = [df_sample["km"].mean(0)] + [df_sample["km"][strata==h].mean(0) for h in range(1,L+1)]
r_var = [0,0,0,0]
for h in range(1,L+1):
    r_var[h] = (1-n[h]/N[h]) * 1/n[h] * 1/x_mean[h]**2 * 1/(n[h]-1) * sum(
               (df_sample["min"][strata==h] - r[h]*df_sample["km"][strata==h])**2)
r_var[0] = sum( (N[h]/N[0])**0.5 * r_var[h] for h in range(1, L+1))
r_std = np.array(r_var)**0.5

# Calculate confidence interval
r_interval = r[0] + r_std[0]*norm.ppf((0.5-confidence/2, 0.5+confidence/2))



# Estimation of mean using strata and ratio
mean = sum(N[h]/N[0] * r[h]*df_population["km"][df_population["strata"]==h].mean(0)
           for h in range(1, L+1))
mean_var = sum( (N[h]/N[0])**2 * df_population["km"][df_population["strata"]==h].mean(0)**2 *
               r_var[h] for h in range(1, L+1))
mean_std = mean_var**0.5
mean_interval = mean + mean_std*norm.ppf((0.5-confidence/2, 0.5+confidence/2))



# Estimation of total using strata and ratio
total = sum(r[h]*df_population["km"][df_population["strata"]==h].sum() for h in range(1, L+1))
total_var = sum(df_population["km"][df_population["strata"]==h].sum()**2 * r_var[h]
                for h in range(1,L+1))
total_std = total_var**0.5
total_interval = total + total_std*norm.ppf((0.5-confidence/2, 0.5+confidence/2))


print("="*100)
print("ESTIMACIÓN CON ESTRATOS Y RAZÓN POR SEPARADO:\n")
print("El estimador de la razón min/km es:", round(r[0],2))
print("Se estima entre:: ",np.round(r_interval,2))
print("\nLa estimación de la media de minutos es:",round(mean,2))
print("Se estima entre:",np.round(mean_interval,2))
print("\nLa estimación del total de minutos es:",round(total,2))
print("Se estima entre:",np.round(total_interval,2))
print("="*100)

ESTIMACIÓN CON ESTRATOS Y RAZÓN POR SEPARADO:

El estimador de la razón min/km es: 5.58
Se estima entre::  [4.84 6.32]

La estimación de la media de minutos es: 117.39
Se estima entre: [111.37 123.41]

La estimación del total de minutos es: 105651.11
Se estima entre: [100229.39 111072.84]


### 4] Estimador de razón combinado
    - Se usa cuando se tienen muchos estratos y/o los tamaños de muestra en cada estrato son pequeños.
    
    - Supone que las razones de cada estrato son similares
    
    - Asume m.a.s. dentro de cada estrato
    
    - La razón se estima como:
        r = sum(N_h*y_mean_h) / sum(N_h*x_mean_h) --- ambas[for h in L]
            donde:
            N_h: Población en el estrato
            y_mean_h: Media muestral en el estrato h de la variable objetivo
            x_mean_h: Media muestral en el estrato h de la variable auxiliar
            
    - Con varianza:
        V(R_s) = 1/X^2 * sum[for h in L](N_h^2 * (1/n_h-1/N_h) * 1/(n_h-1) * ...
                      ...sum[for i in n_h]( ( y_hi - r*x_hi - 1/n_h*...
                      ...sum[for j in n_h](y_hj - r*x_hj) )^2
                      )))
            donde:
            V(r) = Varianza del estimador de la razón
            Nota: Obsérvese que hay sumas anidadas.
            Nota: Es la suma de los cuadrados, no el cuadrado de la suma
            
    - La media poblacional se estima como:
        mean = r * X_mean
            donde:
            mean: Estimador de la media poblacional de la variable objetivo
            r: Estimador de la razón combinado
            X_mean: Media poblacional de la variable auxiliar
            
    - Con varianza:
        V(mean) = X_mean^2 * V(r)
            donde:
            V(mean): Varianza del estimador de la media
            X_mean: Media poblacional de la variable objetivo
            V(r): Varianza del estimador de razón combinado
            
    - El total poblacional se estima como:
        total = r * X
            donde:
            total: Estimador del total poblacional
            r: Estimador de la razón combinada
            X: Total poblacional de la variable auxiliar
            
    - Con varianza:
        V(total) = X^2 * V(r)
            donde:
            V(total): Varianza del estimador de razón
            X: Total poblacional de la variable auxiliar
            V(r): Varianza del estimador de razón combinado

In [9]:
# Compute estimator of ratio
fac_1 = sum(N[h]*df_sample["min"][strata==h].mean(0) for h in range(1, L+1))
fac_2 = sum(N[h]*df_sample["km" ][strata==h].mean(0) for h in range(1, L+1))
r = fac_1 / fac_2

# Compute variance of ratio estimator
r_var = 1/df_population["km"].sum()**2 * sum(N[h]**2 * (1/n[h]-1/N[h]) * 1/(n[h]-1) *
        sum((df_sample["min"][strata==h].iloc[i] - r*df_sample["km"][strata==h].iloc[i] - 1/n[h] *
        sum( df_sample["min"][strata==h].iloc[j] - r*df_sample["km"][strata==h].iloc[j]
        for j in range(n[h])) )**2    
        for i in range(n[h]))
        for h in range(1, L+1))
r_std = r_var**0.5

# Compute confidence interval
r_interval = r + r_std*norm.ppf((0.5-confidence/2, 0.5+confidence/2))



# Compute estimator of mean
mean = r * df_population["km"].mean(0)
mean_var = df_population["km"].mean(0)**2 * r_var
mean_std = mean_var**0.5
mean_interval = mean + mean_std*norm.ppf((0.5-confidence/2, 0.5+confidence/2))



# Compute estimator of total
total = r * df_population["km"].sum(0)
total_var = df_population["km"].sum(0)**2 * r_var
total_std = total_var**0.5
total_interval = total + total_std*norm.ppf((0.5-confidence/2, 0.5+confidence/2))




print("="*100)
print("ESTIMACIÓN CON ESTRATOS Y RAZÓN COMBINADOS:\n")
print("El estimador de la razón min/km es:", round(r,2))
print("Se estima entre:: ",np.round(r_interval,2))
print("\nLa estimación de la media de minutos es:",round(mean,2))
print("Se estima entre:",np.round(mean_interval,2))
print("\nLa estimación del total de minutos es:",round(total,2))
print("Se estima entre:",np.round(total_interval,2))
print("="*100)

ESTIMACIÓN CON ESTRATOS Y RAZÓN COMBINADOS:

El estimador de la razón min/km es: 5.6
Se estima entre::  [5.31 5.89]

La estimación de la media de minutos es: 117.39
Se estima entre: [111.31 123.47]

La estimación del total de minutos es: 105649.44
Se estima entre: [100174.82 111124.06]
